![alvaro-mendoza-6dRiUBjRvsM-unsplash](alvaro-mendoza-6dRiUBjRvsM-unsplash.jpg)

# PFC School Pick Likelihood: Top Programs for Rising Freshman Soccer Players
In supporting Plantation Football Club UPSL players in college preparation, this project is aggregating player data (including performance and academics) and comparing that against the dream, target, and safety schools the player's have indicated on a survey (distributed internally). This project will be conducted in phases with the end goal of providing the top 3 best fit schools (i.e. Junior College, Division 1 - 3), based on aformentioned data sources. Thereafter, this project will be used to spit out insights reports for the team for use in future seasons to improve player management and demonstrate the benefits of PFC UPSL and how it acts as a jump off point to leveling up a player's soccer career.

_PHASE 0: Identifying base key metrics to be acquired from players and schools used by scouts to determine interest_
_PHASE 1: Data capture of player information via survey to club members (UPSL U19s and U17s)_
Building data pipelines from NCAA, NJCAA, and "interested" school websites. (Using an ETL process)
1. Data Wrangling: Website URLs to pull statistics from, additional data to be captured in-house
- Seperated the files by statistic due to issues with parsing a full csv
- Next step is to go back and do it from the larger csv where I splice it without issue
- Detailed statistics can be found here: http://web1.ncaa.org/stats/StatsSrv/rankings
3. Data Cleaning: Transforming data into appropriate fields, types, etc. for analysis
4. Data Storing: Create files to be used in python for future phases and model building

TO-DOs:
- Clean up school tables (e.g. name column to match)
- Get academic information for each of the relevant schools
- Pull in dummy player information from UPSL spreadsheet for time being
- Update with actuals once available
- Create a fuzzy match as a catch all for spelling character mistakes

PHASE 2: Explore the data and determine key data to pull into one table for players and one table for teams
- Data exploration using SQL and the df created in Python in Phase 1
- Formulate foundational insights from school data (e.g. graduating seniors in position, GPA, major)

PHASE 3: Weight information against acceptance rates as a soccer player & stats matching for similar player profile on team
PHASE 4: Adjust and update accordingly to improve accuracy of model (incl. historical training data)
PHASE 5: Automate data capture to pull in total teams in all leagues & run player profile against model to find best fit

## PHASE 1: DATA GATHERING

In [3]:
import pandas as pd

In [4]:
# NCAA Mens Soccer RPI Rankings 2023
url_ncaa_d1_teams = "https://www.ncaa.com/rankings/soccer-men/d1/ncaa-mens-soccer-rpi"

html_ncaa_d1_teams = pd.read_html(url_ncaa_d1_teams)

In [5]:
html_ncaa_d1_teams[0].head()

,Rank,School,Conference,Record,Road,Neutral,Home,Non Div I
0,1,Syracuse,ACC,19-2-4,5-0-2,3-0-0,11-2-2,0-0-0
1,2,Indiana,Big Ten,13-5-6,4-3-3,1-1-0,8-1-3,1-0-0
2,3,Creighton,Big East,13-5-6,5-3-2,2-1-0,6-1-4,0-0-0
3,4,Virginia,ACC,10-4-5,3-0-2,0-1-0,7-3-3,0-0-0
4,5,Washington,Pac-12,15-2-3,5-0-2,0-0-0,10-2-1,0-0-0


In [6]:
rankings_ncaa_d1_teams = html_ncaa_d1_teams[0]

In [7]:
rankings_ncaa_d1_teams.dtypes

Rank           int64
School        object
Conference    object
Record        object
Road          object
Neutral       object
Home          object
Non Div I     object
dtype: object

In [8]:
rankings_ncaa_d1_teams[['Wins', 'Draws', 'Loses']] = rankings_ncaa_d1_teams['Record'].str.split('-', expand=True)

print(rankings_ncaa_d1_teams.head())

   Rank      School Conference  Record  ... Non Div I Wins Draws Loses
0     1    Syracuse        ACC  19-2-4  ...     0-0-0   19     2     4
1     2     Indiana    Big Ten  13-5-6  ...     1-0-0   13     5     6
2     3   Creighton   Big East  13-5-6  ...     0-0-0   13     5     6
3     4    Virginia        ACC  10-4-5  ...     0-0-0   10     4     5
4     5  Washington     Pac-12  15-2-3  ...     0-0-0   15     2     3

[5 rows x 11 columns]


In [9]:
# FIU Website Team Stats 
url_fiu = "https://fiusports.com/sports/mens-soccer/stats/2023"

html_fiu = pd.read_html(url_fiu)

In [10]:
len(html_fiu)

24

In [11]:
#TO DO: Clean up Player Name to "FirstName LastName"
html_fiu[1].head()

,#,Player,GP,GS,MIN,G,A,PTS,SH,SH%,SOG,SOG%,YC-RC,GW,PG-PA,Bio Link
0,9.0,"Mustre, Eduardo 9Mustre, Eduardo",6,6,448,3,1,7,21,0.143,10,0.476,2-0,1,1-1,View Bio
1,4.0,"Skraep, Kasper 4Skraep, Kasper",6,3,379,2,0,4,4,0.500,4,1.000,2-0,0,0-0,View Bio
2,10.0,"Nagao, Yushi 10Nagao, Yushi",6,2,330,1,1,3,5,0.200,2,0.400,2-0,0,0-0,View Bio
3,20.0,"Alves, Paulo 20Alves, Paulo",6,6,432,1,0,2,8,0.125,4,0.500,3-0,1,0-0,View Bio
4,7.0,"Dos Santos Monteiro, Bernardo 7Dos Santos Mon...",6,4,368,1,0,2,14,0.071,6,0.429,1-0,0,0-0,View Bio


In [12]:
#TO DO: Clean up Player Name to "FirstName LastName"
html_fiu[2].head()

,#,Player,GP,GS,MIN,GA,GAA,SV,SV%,W,L,T,SHO/CBO,SF,Bio Link
0,30.0,"Hogan, Tyler 30Hogan, Tyler",5,5,440:32,5,1.02,17,0.773,2.0,1.0,2.0,1/1,38,View Bio
1,28.0,"Sigurjonsson, Vilhelm 28Sigurjonsson, Vilhelm",2,1,99:28,2,1.81,5,0.714,0.0,1.0,0.0,0/1,13,View Bio
2,NaN,Total,6,-,540:00,7,1.17,22,0.759,2.0,2.0,2.0,2,51,NaN
3,NaN,Opponents,6,-,540:00,9,1.50,32,0.780,NaN,NaN,NaN,2,93,NaN


In [13]:
html_fiu[0].head()

,Statistic,No. 11 FIU,Opponents OPP
0,Shots,Shots,Shots
1,Goals G,9,7
2,Goals Per Game G/AVG,1.50,1.17
3,Shots SH,93,51
4,Shots Per Game SH/AVG,15.5,8.5


In [14]:
# Total Team Stats Tables - FIU
team_fiu = html_fiu[0]
off_fiu = html_fiu[1]
gk_fiu = html_fiu[2]

In [15]:
# NCAA D1 Stats - Player Details
# Note: Come back and create a function to parse all files going forward and generate tables
table_ncaa_d1_apg = pd.read_csv("2223_NCAA D1_Assist Per Game.csv", header=3)
table_ncaa_d1_glsagt = pd.read_csv("2223_NCAA D1_Goals Against Avg.csv", header=3)
table_ncaa_d1_gpg = pd.read_csv("2223_NCAA D1_Goals Per Game.csv", header=3)
table_ncaa_d1_spg = pd.read_csv("2223_NCAA D1_Saves Per Game.csv", header=3)
table_ncaa_d1_ttla = pd.read_csv("2223_NCAA D1_Total Assists.csv", header=3, encoding='latin-1')

In [16]:
table_ncaa_d1_apg.head()

,Rank,Name,Team,Cl,Pos,Games,Assists,Per Game
0,1,Nick Gutmann,Kentucky,Sr.,M,21.0,19.0,0.90
1,2,Tom Cooklin,George Washington,Sr.,F,17.0,13.0,0.76
2,3,Willem Ebbinge,Harvard,Jr.,M,16.0,11.0,0.69
3,4,JC Ngando,UNC Greensboro,So.,M,21.0,14.0,0.67
4,5,Alexander Petraeus,SMU,Fr.,F,16.0,10.0,0.63


In [17]:
table_ncaa_d1_glsagt.head()

,Rank,Name,Team,Cl,Pos,Games,Goalie Min. Plyd,GA,GAA
0,1,Hunter Morse,Western Mich.,Sr.,GK,21.0,1907:57,10.0,0.472
1,2,Danny DiMarco,Manhattan,Jr.,GK,14.0,1200:48,7.0,0.525
2,3,Eliot Hamill,Duke,Sr.,GK,19.0,1710:00,11.0,0.579
3,4,Lute Lillo Portero,Omaha,Jr.,GK,9.0,765:00,5.0,0.588
4,5,Jassem Koleilat,New Hampshire,Sr.,GK,20.0,1802:58,12.0,0.599


In [18]:
table_ncaa_d1_gpg.head()

,Rank,Name,Team,Cl,Pos,Games,Goals,Per Game
0,1,Duncan McGuire,Creighton,Jr.,F,24.0,23.0,0.96
1,2,Lucca Dourado,UCF,Jr.,F,13.0,11.0,0.85
2,3,Eliot Goldthorp,Hofstra,So.,M,21.0,17.0,0.81
3,4,Emil Jaaskelainen,LIU,Jr.,F,15.0,12.0,0.80
4,5,Shaun Joash,Grand Canyon,Sr.,F,16.0,12.0,0.75


In [19]:
table_ncaa_d1_spg.head()

,Rank,Name,Team,Cl,Pos,Games,Goalie Min. Plyd,Saves,Per Game
0,1,Rotem Fadida,Hartford,Jr.,GK,12.0,1022:54,78.0,6.50
1,2,Michael Sly Jr.,Oakland,So.,GK,17.0,1458:36,98.0,5.76
2,3,Luca Marinelli,Bryant,So.,GK,12.0,1080:00,69.0,5.75
3,4,Jacob Heihsel,UMass Lowell,Sr.,GK,12.0,1080:00,68.0,5.67
4,5,Gabe Calhoun,VMI,Sr.,GK,15.0,1350:00,84.0,5.60


In [20]:
table_ncaa_d1_ttla.head()

,Rank,Name,Team,Cl,Pos,Games,Assists
0,1,Nick Gutmann,Kentucky,Sr.,M,21.0,19.0
1,2,JC Ngando,UNC Greensboro,So.,M,21.0,14.0
2,NaN,Giorgio Probo,Creighton,So.,M,24.0,14.0
3,4,Tom Cooklin,George Washington,Sr.,F,17.0,13.0
4,NaN,Alexander Stjernegaard,Quinnipiac,Fr.,D,21.0,13.0


In [21]:
# UPSL Plantation FC Division 1 and 2 Team Details + Player Stats
import requests

# Team Standings
url_upsl_prem = requests.get("https://premier.upsl.com/standings/").content
url_upsl_div = requests.get("https://division1.upsl.com/standings/").content

html_upsl_prem = pd.read_html(url_upsl_prem)
html_upsl_div = pd.read_html(url_upsl_div)

In [22]:
len(html_upsl_prem)

26

In [23]:
len(html_upsl_div)

21

In [24]:
# Premiere League: Plantation FC
html_upsl_prem[6]

,P,Team,GP,W,D,L,H (W-D-L),A (W-D-L),Latest,GF,GA,GD,PTSP
0,1,Little Haiti FC,4,4,0,0,3 - 0 - 0,1 - 0 - 0,W W W W,18,5,13,12
1,2,Plantation FC,2,2,0,0,2 - 0 - 0,0 - 0 - 0,W W,7,2,5,6
2,3,Inter Miami CF Academy,2,2,0,0,2 - 0 - 0,0 - 0 - 0,W W,6,2,4,6
3,4,ISA,3,2,0,1,1 - 0 - 0,1 - 0 - 1,W W L,6,5,1,6
4,5,Athletum Miami FC U19,3,2,0,1,1 - 0 - 1,1 - 0 - 0,W L W,6,6,0,6
5,6,Dade County FC,3,1,1,1,1 - 1 - 1,0 - 0 - 0,W D L,7,5,2,4
6,7,City Soccer FC,4,1,1,2,0 - 0 - 0,1 - 1 - 2,D W L L,5,11,-6,4
7,8,Rush Select Academy,1,1,0,0,0 - 0 - 0,1 - 0 - 0,W,4,2,2,3
8,9,Hodler Miami Fc,3,1,0,2,0 - 0 - 1,1 - 0 - 1,L W L,6,5,1,3
9,10,Parkland Soccer Club,2,1,0,1,1 - 0 - 1,0 - 0 - 0,L W,3,3,0,3


In [25]:
# U19 League: Plantation FC II
html_upsl_div[7]

,P,Team,GP,W,D,L,H (W-D-L),A (W-D-L),Latest,GF,GA,GD,PTSP
0,1,Weston FC U19 Black,1,1,0,0,0 - 0 - 0,1 - 0 - 0,W,3,2,1,3
1,2,Island United AC,1,1,0,0,0 - 0 - 0,1 - 0 - 0,W,3,2,1,3
2,3,Plantation FC II,1,0,1,0,0 - 1 - 0,0 - 0 - 0,D,0,0,0,1
3,4,South Florida Football Academy,2,0,1,1,0 - 0 - 1,0 - 1 - 0,D L,2,3,-1,1
4,5,Athletum Miami FC U17,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
5,6,Springs Soccer Club,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
6,7,Milan De Miami FC,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
7,8,Parkland Soccer Club II,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
8,9,Atletico de Madrid Academia Miami,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
9,10,Empire SC,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0


In [26]:
# U17 League: Plantation FC III
html_upsl_div[8]

,P,Team,GP,W,D,L,H (W-D-L),A (W-D-L),Latest,GF,GA,GD,PTSP
0,1,Rush Select II,1,1,0,0,0 - 0 - 0,1 - 0 - 0,W,4,1,3,3
1,2,Weston FC U18 Black,1,0,1,0,0 - 1 - 0,0 - 0 - 0,D,2,2,0,1
2,3,Milan de Miami Academy,1,0,1,0,0 - 0 - 0,0 - 1 - 0,D,2,2,0,1
3,4,Athletum FC U19 Elite,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
4,5,Hodler Miami FC II,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
5,6,South Florida Football Academy II,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
6,7,Naples City FC D.1,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
7,8,FBS FC,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
8,9,PSG Academy Florida,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0
9,10,Plantation FC III,0,0,0,0,0 - 0 - 0,0 - 0 - 0,NaN,0,0,0,0


In [27]:
# Plantation FC: Individual Player & Game Details
url_upsl_pfcp = requests.get("https://premier.upsl.com/teams/plantation-fc-241834/").content
url_upsl_pfcd1 = requests.get("https://division1.upsl.com/teams/plantation-fc-ii-39664/").content
url_upsl_pfcd2 = requests.get("https://division1.upsl.com/teams/plantation-fc-iii-90727/").content

html_upsl_pfcp = pd.read_html(url_upsl_pfcp)
html_upsl_pfcd1 = pd.read_html(url_upsl_pfcd1)
html_upsl_pfcd2 = pd.read_html(url_upsl_pfcd2)

In [28]:
len(html_upsl_pfcp)

1

In [29]:
html_upsl_pfcp[0]

,Home,Away,Date,Time,H,A,Venue,Unnamed: 7
0,Plantation FC,Rush Select Academy,10/16/2023,4:00 pm PT,0,0,UPSL Stadium,upsl__show__schedule
1,Plantation FC,Weston FC,09/08/2023,6:00 pm PT,2,1,UPSL Stadium,upsl__show__results
2,Hodler Miami Fc,Plantation FC,09/13/2023,4:00 pm PT,0,0,UPSL Stadium,upsl__show__results
3,Plantation FC,Athletum Miami FC U19,10/20/2023,6:00 pm PT,0,0,UPSL Stadium,upsl__show__schedule
4,Palm Beach Flames SC,Plantation FC,09/17/2023,4:00 pm PT,0,0,UPSL Stadium,upsl__show__schedule
5,Inter Miami CF Academy,Plantation FC,09/29/2023,4:00 pm PT,0,0,Inter Miami Training Center,upsl__show__schedule
6,Plantation FC,Florida Soccer Soldiers,10/06/2023,6:15 pm PT,0,0,UPSL Stadium,upsl__show__schedule
7,Plantation FC,Miami Rush Kendall SC,11/08/2023,6:00 pm PT,0,0,UPSL Stadium,upsl__show__schedule
8,Plantation FC,City Soccer FC,09/13/2023,4:00 pm PT,5,1,UPSL Stadium,upsl__show__results
9,Parkland Soccer Club,Plantation FC,11/17/2023,6:00 pm PT,0,0,UPSL Stadium,upsl__show__schedule


In [30]:
# UPSL PFC Player Questionnaire Responses

table_survey = pd.read_csv("UPSL College Prep - Responses_ Player Profile CV.csv")

In [31]:
table_survey.head()

,Timestamp,Email Address,Current Team,Player's Full Name,Player's Address,Contact Email,Right handed or left handed?,Contact Number (###-###-####),Position,Jersey #,Birthdate,Height in feet and inches,Weight in lbs (i.e. 178),Parents Name(s),Graduation Year,Grade Rank / # of Students (i.e. 50 / 225),Graduating High School,City of Graduating High School,Additional High School (if transferred),City of Additional High School (if transferred),GPA (i.e. 3.3) Note: Transcripts will need to be verified.,Is your GPA weighted or unweighted?,Did you make Honor Roll?,"If yes to Honor Roll, what grade(s) and year(s)?",AP Score(s),SAT or ACT Score,SAT or ACT Date,School Clubs / Memberships / Extracurriculars,What type of college do you want to go to? Check all that apply.,"What do you want to study in college? If you do not want to go to college, write ""N/A"".",What are your dream schools? (List 2),What are your target schools? (List 2),What is your safety school? (List 2),Have you registered with the NCAA?,"If yes, what is your number?",List any club tournaments or playoffs or school championships you won (include year or age group),"Did you ever play up a year? If so, when and what team?","Did you get invited to or attend a National ID camp or ODP? If so, what type, when or age group?","Did you get picked for First Team All-County or Honorable Mention? If so, when and what County?",List any other athletic achievements you have:,Club 1: Name,Club 1: Age Group,Club 2: Name (if applicable),Club 2: Age Group (if applicable),Club 3: Name (if applicable),Club 3: Age Group (if applicable),Club 4: Name (if applicable),Club 4: Age Group (if applicable),Coach 1: Name,Coach 1: Phone #,Coach 1: Club,Coach 1: Age Group,Coach 2: Name (if applicable),Coach 2: Phone # (if applicable),Coach 2: Club (if applicable),Coach 2: Age Group (if applicable),Coach 3: Name (if applicable),Coach 3: Phone # (if applicable),Coach 3: Club (if applicable),Coach 3: Age Group (if applicable),Do you have a recent video?,"If yes, please add the link:",Club 1: Age Group.1
0,9/12/2023 18:44:30,alissa.b.deltoro@gmail.com,UPSL - U19,Alissa Del Toro,"100 South Laurel Drive Margate, FL 33063",alissa.b.deltoro@gmail.com,Right Handed,3019100528,Goalie,12,4/25/1991,5'3,178,Wanda Del Toro,2023,20/113,Cardinal Gibbons High School,Fort Lauderdale,NaN,NaN,4.1,Weighted,Yes,2005 - 2009,NaN,1500,1/1/2000,All of them!,"Division 2 (D2), Division 3 (D3)",Aerospace Engineering,Cornell,Harvard,RPI,No,NaN,Blah,Blah blah,Blah blah blah,NaN,NaN,Plantation FC,UPSL,NaN,NaN,NaN,NaN,NaN,NaN,Anis,444-444-4444,Plantation FC,Option 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN


## PHASE 2: DATA EXPLORATION (IN SQL)

In [32]:
-- Query a variable:
-- SELECT * FROM df
--
-- Query a CSV file:
-- SELECT * FROM 'data.csv'

SELECT Pos, COUNT(*) FROM table_ncaa_d1_ttla WHERE Cl = 'Sr.' GROUP BY Pos


,Pos,count_star()
0,M,33
1,F,25
2,D,10


In [33]:
SELECT * FROM table_ncaa_d1_ttla WHERE Cl = 'Sr.' AND Pos = 'M'

,Rank,Name,Team,Cl,Pos,Games,Assists
0,1,Nick Gutmann,Kentucky,Sr.,M,21.0,19.0
1,9,Ryan Becher,UMBC,Sr.,M,18.0,10.0
2,NaN,Dylan Gaither,Mercer,Sr.,M,19.0,10.0
3,NaN,Stephen Hasse,Niagara,Sr.,M,20.0,9.0
4,NaN,Leopoldo Hernandez,UC Riverside,Sr.,M,20.0,9.0
5,NaN,Alex Nagy,Vermont,Sr.,M,17.0,9.0
6,NaN,Jacob Angeletti,Air Force,Sr.,M,17.0,8.0
7,NaN,Malcolm Johnston,Maryland,Sr.,M,17.0,8.0
8,NaN,Alexander Levengood,William & Mary,Sr.,M,18.0,8.0
9,NaN,Lucas Meek,Washington,Sr.,M,20.0,8.0


In [34]:
SELECT * FROM table_ncaa_d1_ttla WHERE Cl = 'Sr.' AND Pos = 'F'

,Rank,Name,Team,Cl,Pos,Games,Assists
0,4,Tom Cooklin,George Washington,Sr.,F,17.0,13.0
1,7,Eric Conerty,Western Mich.,Sr.,F,21.0,11.0
2,NaN,Ryan Wittenbrink,Indiana,Sr.,F,24.0,9.0
3,28,Marco Afonso,UNC Greensboro,Sr.,F,21.0,8.0
4,NaN,Sergi Martinez,Bowling Green,Sr.,F,16.0,8.0
5,53,Brage Aasen,Quinnipiac,Sr.,F,12.0,7.0
6,NaN,David Bercedo,Quinnipiac,Sr.,F,21.0,7.0
7,NaN,Emeka Eneli,Cornell,Sr.,F,17.0,7.0
8,NaN,Daniel Hernandez,Campbell,Sr.,F,20.0,7.0
9,NaN,Bryan Iliohan,California Baptist,Sr.,F,21.0,7.0


In [35]:
SELECT * FROM table_ncaa_d1_ttla WHERE Cl = 'Sr.' AND Pos = 'D'

,Rank,Name,Team,Cl,Pos,Games,Assists
0,NaN,Noah Gulden,Lipscomb,Sr.,D,19.0,10.0
1,NaN,Connor Drought,Cornell,Sr.,D,18.0,8.0
2,NaN,Nathan Simeon,San Francisco,Sr.,D,17.0,8.0
3,NaN,Hayden Anderson,Memphis,Sr.,D,19.0,7.0
4,NaN,Brendan Clark,UC Riverside,Sr.,D,20.0,7.0
5,NaN,Matt Lockwood,Western Mich.,Sr.,D,21.0,7.0
6,NaN,Nyk Sessock,Indiana,Sr.,D,24.0,7.0
7,NaN,Keegan Tingey,Stanford,Sr.,D,20.0,7.0
8,NaN,Omar Grey,Washington,Sr.,D,18.0,6.0
9,NaN,Kasper Lehm,Elon,Sr.,D,16.0,6.0


In [39]:
SELECT Pos, COUNT(*) FROM table_ncaa_d1_apg WHERE Cl = 'Sr.' GROUP BY Pos

,Pos,count_star()
0,M,34
1,F,23
2,D,9


In [43]:
SELECT * FROM table_ncaa_d1_apg WHERE Cl = 'Sr.' AND Pos = 'M'

,Rank,Name,Team,Cl,Pos,Games,Assists,Per Game
0,1,Nick Gutmann,Kentucky,Sr.,M,21.0,19.0,0.90
1,11,Ryan Becher,UMBC,Sr.,M,18.0,10.0,0.56
2,NaN,Alex Nagy,Vermont,Sr.,M,17.0,9.0,0.53
3,15,Dylan Gaither,Mercer,Sr.,M,19.0,10.0,0.53
4,NaN,Koby Carr,Charlotte,Sr.,M,14.0,7.0,0.50
5,28,Jacob Angeletti,Air Force,Sr.,M,17.0,8.0,0.47
6,NaN,Malcolm Johnston,Maryland,Sr.,M,17.0,8.0,0.47
7,NaN,Stephen Hasse,Niagara,Sr.,M,20.0,9.0,0.45
8,NaN,Leopoldo Hernandez,UC Riverside,Sr.,M,20.0,9.0,0.45
9,NaN,Alexander Levengood,William & Mary,Sr.,M,18.0,8.0,0.44


In [44]:
SELECT * FROM table_ncaa_d1_apg WHERE Cl = 'Sr.' AND Pos = 'F'

,Rank,Name,Team,Cl,Pos,Games,Assists,Per Game
0,2,Tom Cooklin,George Washington,Sr.,F,17.0,13.0,0.76
1,18,Eric Conerty,Western Mich.,Sr.,F,21.0,11.0,0.52
2,NaN,Sergi Martinez,Bowling Green,Sr.,F,16.0,8.0,0.50
3,NaN,Gino Vivi,UCF,Sr.,F,10.0,5.0,0.50
4,NaN,Kameron Lacey,Charlotte,Sr.,F,16.0,7.0,0.44
5,NaN,Emeka Eneli,Cornell,Sr.,F,17.0,7.0,0.41
6,NaN,Diba Nwegbo,William & Mary,Sr.,F,17.0,7.0,0.41
7,NaN,Demitrius Kigeya,Gonzaga,Sr.,F,18.0,7.0,0.39
8,77,Marco Afonso,UNC Greensboro,Sr.,F,21.0,8.0,0.38
9,80,Sebastian Andreassen,Central Ark.,Sr.,F,16.0,6.0,0.38


In [45]:
SELECT * FROM table_ncaa_d1_apg WHERE Cl = 'Sr.' AND Pos = 'D'

,Rank,Name,Team,Cl,Pos,Games,Assists,Per Game
0,NaN,Noah Gulden,Lipscomb,Sr.,D,19.0,10.0,0.53
1,NaN,Nathan Simeon,San Francisco,Sr.,D,17.0,8.0,0.47
2,41,Connor Drought,Cornell,Sr.,D,18.0,8.0,0.44
3,NaN,Kasper Lehm,Elon,Sr.,D,16.0,6.0,0.38
4,89,Hayden Anderson,Memphis,Sr.,D,19.0,7.0,0.37
5,101,Brendan Clark,UC Riverside,Sr.,D,20.0,7.0,0.35
6,NaN,Keegan Tingey,Stanford,Sr.,D,20.0,7.0,0.35
7,NaN,Omar Grey,Washington,Sr.,D,18.0,6.0,0.33
8,NaN,Matt Lockwood,Western Mich.,Sr.,D,21.0,7.0,0.33


In [40]:
SELECT Pos, COUNT(*) FROM table_ncaa_d1_glsagt WHERE Cl = 'Sr.' GROUP BY Pos

,Pos,count_star()
0,GK,71
1,-,1


In [46]:
SELECT * FROM table_ncaa_d1_glsagt WHERE Cl = 'Sr.' AND Pos != 'GK'

,Rank,Name,Team,Cl,Pos,Games,Goalie Min. Plyd,GA,GAA
0,NaN,Michael Stone,UConn,Sr.,-,11.0,990:00,14.0,1.273


In [ ]:
# Clean up Goals Against Table: Check on Pos '-' and fill in as 'GK' in dataframe
#####



In [41]:
SELECT Pos, COUNT(*) FROM table_ncaa_d1_gpg WHERE Cl = 'Sr.' GROUP BY Pos

,Pos,count_star()
0,F,50
1,M,20
2,D,4


In [47]:
SELECT * FROM table_ncaa_d1_gpg WHERE Cl = 'Sr.' AND Pos = 'F'

,Rank,Name,Team,Cl,Pos,Games,Goals,Per Game
0,5,Shaun Joash,Grand Canyon,Sr.,F,16.0,12.0,0.75
1,NaN,Stefan Sigurdarson,Boston College,Sr.,F,16.0,12.0,0.75
2,9,David Bercedo,Quinnipiac,Sr.,F,21.0,14.0,0.67
3,12,Finn Ballard McBride,UC Santa Barbara,Sr.,F,20.0,13.0,0.65
4,NaN,Jelldrik Dallmann,Lipscomb,Sr.,F,17.0,11.0,0.65
5,18,MD Myers,Rutgers,Sr.,F,21.0,13.0,0.62
6,24,Kameron Lacey,Charlotte,Sr.,F,16.0,9.0,0.56
7,26,Liam Butts,Penn St.,Sr.,F,13.0,7.0,0.54
8,27,Paolo Carroll,Yale,Sr.,F,15.0,8.0,0.53
9,32,Stephen Afrifa,FIU,Sr.,F,19.0,10.0,0.53


In [48]:
SELECT * FROM table_ncaa_d1_gpg WHERE Cl = 'Sr.' AND Pos = 'M'

,Rank,Name,Team,Cl,Pos,Games,Goals,Per Game
0,7,Ryan Becher,UMBC,Sr.,M,18.0,13.0,0.72
1,8,Knut Ahlander,SMU,Sr.,M,17.0,12.0,0.71
2,NaN,Kieran Richards,ETSU,Sr.,M,17.0,11.0,0.65
3,NaN,Ben Stitz,Penn,Sr.,M,18.0,11.0,0.61
4,NaN,Alex Nagy,Vermont,Sr.,M,17.0,9.0,0.53
5,NaN,Valentin Noel,Pittsburgh,Sr.,M,21.0,11.0,0.52
6,NaN,Charlie Sharp,Western Mich.,Sr.,M,21.0,11.0,0.52
7,NaN,Lucas Meek,Washington,Sr.,M,20.0,10.0,0.50
8,65,Davi Alves,FGCU,Sr.,M,15.0,7.0,0.47
9,NaN,Pablo Kawecki,Cleveland St.,Sr.,M,20.0,9.0,0.45


In [49]:
SELECT * FROM table_ncaa_d1_gpg WHERE Cl = 'Sr.' AND Pos = 'D'

,Rank,Name,Team,Cl,Pos,Games,Goals,Per Game
0,76,Ryan Clare,Princeton,Sr.,D,16.0,7.0,0.44
1,NaN,Daniel Nimick,Western Mich.,Sr.,D,21.0,8.0,0.38
2,NaN,Timo Hummrich,Fordham,Sr.,D,16.0,6.0,0.38
3,139,Lewis Green,Missouri St.,Sr.,D,14.0,5.0,0.36


In [42]:
SELECT Pos, COUNT(*) FROM table_ncaa_d1_spg WHERE Cl = 'Sr.' GROUP BY Pos

,Pos,count_star()
0,GK,69


In [50]:
SELECT * FROM table_ncaa_d1_spg WHERE Cl = 'Sr.'

,Rank,Name,Team,Cl,Pos,Games,Goalie Min. Plyd,Saves,Per Game
0,4,Jacob Heihsel,UMass Lowell,Sr.,GK,12.0,1080:00,68.0,5.67
1,5,Gabe Calhoun,VMI,Sr.,GK,15.0,1350:00,84.0,5.60
2,9,Joseba Incera,Radford,Sr.,GK,11.0,990:00,60.0,5.45
3,10,William Howard,Canisius,Sr.,GK,17.0,1530:00,90.0,5.29
4,14,Christian Garner,Northwestern,Sr.,GK,13.0,1170:00,64.0,4.92
...,...,...,...,...,...,...,...,...,...
64,NaN,Russell Shealy,Syracuse,Sr.,GK,24.0,2220:00,72.0,3.00
65,NaN,Alexandre Durand,Longwood,Sr.,GK,16.0,1291:56,47.0,2.94
66,150,Carlos Tofern,Saint Louis,Sr.,GK,17.0,1545:00,49.0,2.88
67,NR,Dominic Peters,UC San Diego,Sr.,GK,17.0,1395:00,60.0,3.53
